# Building a Supercharged Web Agent
We’ll be building a “Supercharged Research Agent” that takes a user prompt, searches the web, and returns a clean, structured summary for human use.

In [7]:
# Import Dependency Libs
from dotenv import load_dotenv
from agents import Agent, WebSearchTool, trace, Runner
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown


load_dotenv(override=True)

# Agent Instructions: Clear and Direct
RESEARCH_PROMPT = (
    "You are a highly efficient Bitech research assistant. "
    "Given a topic, perform a thorough web search and return a well-organized, factual Lecture summary "
    "in under 1000 words. Summarize clearly, capture core insights, and omit fluff. "
    "No personal opinions or commentary. This is for someone writing a Lecture to deliver."
)

# Initialize Web Search Agent
web_search_agent = Agent(
    name="Web Research Agent",
    instructions=RESEARCH_PROMPT,
    tools=[WebSearchTool(search_context_size='low')],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")
)

## Planning Web Searches with Structured AI Output

In [8]:
from pydantic import BaseModel

# Define structured output for each search item
class SearchTask(BaseModel):
    reason: str  # Why this particular search matters
    query: str   # The actual web search string

# The full output schema from the planner
class SearchPlan(BaseModel):
    searches: list[SearchTask]  # A list of search instructions

# How many searches should the planner generate?
NUMBER_OF_SEARCHES=3

# Instructions: Well-formed prompt for planning
PLANNER_PROMPT = (
    f"You are a BioTech Lecture planner. Given a user query, identify {NUMBER_OF_SEARCHES} distinct search terms "
    "that would help answer it comprehensively. For each search, include a short reason explaining its relevance."
)

# Agent to generate the structured search plan
search_planner_agent = Agent(
    name="SearchPlannerAgent",
    instructions=PLANNER_PROMPT,
    model="gpt-4o-mini",
    output_type=SearchPlan  # Agent is expected to output a Pydantic structure
)


## Designing End-to-End Research Pipelines

In [9]:
## This agent takes results from all search tasks and writes a long-form, structured markdown report.

class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final Lecture in Word file format."""

    follow_up_questions: list[str]
    """Suggested topics to research further"""

# Report Generator Agent
INSTRUCTIONS = (
    "You are a senior Lecturer tasked with writing a cohesive report for a given query. "
    "You will be provided with the original query, and some initial research done by a Lecturer assistant.\n"
    "You should first come up with an outline for the Lecturer that describes the structure, details, examples and "
    "flow charts and images in the Lecturer. Then, generate the Lecturer and return that as your final output.\n"
    "The final output should be in Wordfile format, and it should be lengthy and detailed. Aim "
    "for 3–4 pages of content."
)

writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData
)

## Async Pipeline Functions

In [10]:
## First generate a reasoned plan before searching.
import asyncio
async def search_planner(query: str):
    """Use planner_agent to generate a structured list of web searches for a given research query."""
    print("Planning for searches...")
    result = await Runner.run(search_planner_agent, query)
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

## Function to Perform WebSearch : OpenAI WebSearch Tool
async def web_search(search_task: SearchTask):
    """Run a single web search using the search_agent for a given query and reason."""
    input = f"Search term: {search_task.query}\nReason for searching: {search_task.reason}"
    result = await Runner.run(web_search_agent, input)
    return result.final_output

## Running web_search in parallel
async def perform_parallel_searches(search_plan: SearchPlan):
    """Perform web searches in parallel using asyncio to speed up data collection."""
    print("Perform Parallel Searching...")
    tasks = [asyncio.create_task(web_search(task)) for task in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching !!!")
    return results

In [11]:
## Generate Report from Search Results
async def generate_detailed_report(user_query: str, search_summaries: list[str]):
    """Use the WriterAgent to compile a structured, markdown-based research report."""
    print("Thinking deeply about the report...")
    prompt_input = f"Original query: {user_query}\nSummarized search results: {search_summaries}"
    result = await Runner.run(writer_agent, prompt_input)
    print("Finished writing the report!")
    return result.final_output


In [12]:
# Final Pipeline Execution

# Research Query
research_query = "Endergonic vs exergonic processes with examples"



# Step 1: Plan web searches
search_plan = await search_planner(research_query)

# Step 2: Perform all searches asynchronously
search_results = await perform_parallel_searches(search_plan)

# Step 3: Generate structured markdown report
final_report = await generate_detailed_report(research_query, search_results)
print(final_report)


print("🎉 Research workflow completed! Check your inbox.")

Planning for searches...
Will perform 3 searches
Perform Parallel Searching...
Finished searching !!!
Thinking deeply about the report...
Finished writing the report!
short_summary='The report clearly delineates the differences between exergonic and endergonic processes, detailing their roles in chemical thermodynamics, examples from biological systems, and their importance in metabolic pathways.' markdown_report='# Endergonic vs Exergonic Processes\n\n## Outline\n1. **Introduction**  \n   - Definition of thermodynamics in chemical reactions\n   - Importance of understanding energy changes\n\n2. **Exergonic Reactions**  \n   1. Definition and characteristics  \n   2. Equation relating Gibbs free energy  \n   3. Examples  \n      - Cellular Respiration  \n      - Combustion of Glucose  \n      - Hydrolysis of ATP  \n      - Neutralization Reactions  \n   4. Importance in Biological Systems  \n      - Energy production and metabolic pathways  \n\n3. **Endergonic Reactions**  \n   1. Defi

<table style="margin: 0; text-align: left; width:100%"> <tr> <td> <h2 style="color:#00cc00;">🌟 Congratulations!</h2> <span style="color:#00cc00;"> You’ve just designed, built, and executed a complete **Autonomous AI Research Pipeline**.<br/><br/> You didn’t just write prompts—you orchestrated agents, async tasks, and output delivery with production-level patterns.<br/><br/> 🧠 You’ve upskilled.<br/> 💼 You’ve unlocked automation possibilities.<br/> 🛠️ You’ve created reusable agent components.<br/><br/> If you're enjoying this journey, I’d love your support:<br/> 👉 Rate the course on Udemy if you haven’t already<br/> 👉 <a href="https://www.linkedin.com/in/anshulchauhan/">Connect with me on LinkedIn</a><br/></span> </td> </tr> </table>